In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from gensim.parsing.preprocessing import strip_non_alphanum, strip_punctuation, strip_multiple_whitespaces, preprocess_string
import numpy as np
import os
import unicodedata
import dotenv

c:\Users\enaga\miniconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
dataPATH = 'C:/Users/enaga/Proyectos/gerty-homologacion/gerty-homologaciones-ML/experimentos_chile/healthatom/data.xlsx'
provPATH = 'C:/Users/enaga/Proyectos/gerty-homologacion/gerty-homologaciones-ML/experimentos_chile/healthatom/provision.xlsx'

In [3]:
import pandas as pd

data = pd.read_excel(dataPATH)
prov = pd.read_excel(provPATH)

In [7]:
prov

,id,dm_id,specialty,code,name
0,1,1,Acciones Generales,1001,"Examen inicial, plan de tratamiento y presupuesto"
1,2,2,Acciones Generales,1002,Control y examen de rigor
2,3,3,Acciones Generales,1003,Interconsulta con informe escrito (1 sesión)
3,4,4,Acciones Generales,1004,Urgencias: tratamiento inicial (1 sesión)
4,5,7,Acciones Generales,1007,"Estudio preliminar clínico, radiográfico y de ..."
...,...,...,...,...,...
613,614,813,Operatoria,5068,Aplicación sellantes fotocurado pieza definiti...
614,615,814,Trastornos Temporomandibulares,27018,Confección de Dispositivo Intraoral (Trastorno...
615,616,815,Ortodoncia,12061,Instalación quirúrgica de dispositivos de ancl...
616,617,816,Ortodoncia,12920,Laboratorio: Dispositivo de Anclaje esqueletal...


In [8]:
data = data.drop_duplicates()
data

,provision,provider_provision,provision_specialty,provider_provision_specialty
0,Urgencia: Tratamiento inicial (1 sesión),BC01004 Urgencia Tratamiento Inicial 1 Sesión,Urgencias,General
1,Blanqueamiento dentario con Laser (ambas arcad...,Blanqueamiento Laser sin cobertura,Operatoria,General
2,Recementación de incrustación sin correcciones,BC07031 Recementación incrustación sin correcc...,Operatoria,Operatoria
3,Recementación de incrustación con ajuste opera...,BC07032 Recementación de incrustación con ajus...,Operatoria,Operatoria
4,Carillas directas,BC07044 Carillas Directas,Operatoria,Operatoria
...,...,...,...,...
83448,Protesis fija sobre implante atornillada o cem...,Plural sobre implante 3 estructuras cerámica,Prótesis Fija,REHABILITACIÓN SOBRE IMPLANTE
83449,Intermediario dento o implantosoportada en Pró...,Plural sobre implante 3 estructuras cerámica,Prótesis Fija,REHABILITACIÓN SOBRE IMPLANTE
83450,Laboratorio: Intermediario prótesis fija sobre...,Plural sobre implante 3 estructuras cerámica,Implantología,REHABILITACIÓN SOBRE IMPLANTE
83451,"Plastía de rebordes alveolares, paladar duro, ...",Retiro de implante (con osteotomía),Cirugía Maxilofacial,ACCIONES IMPLANTOLOGIA


In [10]:
prov = prov.rename(columns={'specialty': 'provision_specialty'})
prov = prov.rename(columns={'name': 'provision'})
data_code = pd.merge(data, prov, how='left', on=['provision', 'provision_specialty'])
data_code = data_code.drop(['id', 'dm_id', 'provision_specialty'], axis=1)
data_code

,provision,provider_provision,provider_provision_specialty,code
0,Urgencia: Tratamiento inicial (1 sesión),BC01004 Urgencia Tratamiento Inicial 1 Sesión,General,35014
1,Blanqueamiento dentario con Laser (ambas arcad...,Blanqueamiento Laser sin cobertura,General,5066
2,Recementación de incrustación sin correcciones,BC07031 Recementación incrustación sin correcc...,Operatoria,5019
3,Recementación de incrustación con ajuste opera...,BC07032 Recementación de incrustación con ajus...,Operatoria,5020
4,Carillas directas,BC07044 Carillas Directas,Operatoria,5028
...,...,...,...,...
45310,Protesis fija sobre implante atornillada o cem...,Plural sobre implante 3 estructuras cerámica,REHABILITACIÓN SOBRE IMPLANTE,6037
45311,Intermediario dento o implantosoportada en Pró...,Plural sobre implante 3 estructuras cerámica,REHABILITACIÓN SOBRE IMPLANTE,6025
45312,Laboratorio: Intermediario prótesis fija sobre...,Plural sobre implante 3 estructuras cerámica,REHABILITACIÓN SOBRE IMPLANTE,20913
45313,"Plastía de rebordes alveolares, paladar duro, ...",Retiro de implante (con osteotomía),ACCIONES IMPLANTOLOGIA,16049


In [12]:
lista_ninos=['12011', '2002', '2004', '3005', '3006', '13001', '13002'
             , '13003', '13004', '13005', '13006', '13007', '13008', 
             '13009', '13010', '13011', '13012', '13013', '13014', 
             '13015', '13016', '13017', '13018', '13019', '13020', 
             '13021', '13022', '13023', '13024', '13025', '13026', 
             '13027', '13028', '13029', '14001', '14002', '14003', '14004', 
             '14005', '14006', '14007', '14008', '14009', '14010', '14011',
               '14012', '14013', '14014', '14015', '14016', '14017', '14018', 
               '14019', '14020', '14021', '14022', '14023', '14024', '14025',
                 '14026', '14027', '14028', '14029', '14030', '14031', '14032',
                   '14033', '14034', '14035', '14036', '14037']


filtrado_ninos=data_code[data_code['code'].isin(lista_ninos)]
filtrado_adultos=data_code[~data_code['code'].isin(lista_ninos)]
filtrado_ninos['edad']='niño'
filtrado_adultos['edad']='adulto'

In [28]:
df=pd.concat([filtrado_ninos, filtrado_adultos], axis=0)

In [29]:
df

,provision,provider_provision,provider_provision_specialty,code,edad
0,Urgencia: Tratamiento inicial (1 sesión),BC01004 Urgencia Tratamiento Inicial 1 Sesión,General,35014,adulto
1,Blanqueamiento dentario con Laser (ambas arcad...,Blanqueamiento Laser sin cobertura,General,5066,adulto
2,Recementación de incrustación sin correcciones,BC07031 Recementación incrustación sin correcc...,Operatoria,5019,adulto
3,Recementación de incrustación con ajuste opera...,BC07032 Recementación de incrustación con ajus...,Operatoria,5020,adulto
4,Carillas directas,BC07044 Carillas Directas,Operatoria,5028,adulto
...,...,...,...,...,...
45310,Protesis fija sobre implante atornillada o cem...,Plural sobre implante 3 estructuras cerámica,REHABILITACIÓN SOBRE IMPLANTE,6037,adulto
45311,Intermediario dento o implantosoportada en Pró...,Plural sobre implante 3 estructuras cerámica,REHABILITACIÓN SOBRE IMPLANTE,6025,adulto
45312,Laboratorio: Intermediario prótesis fija sobre...,Plural sobre implante 3 estructuras cerámica,REHABILITACIÓN SOBRE IMPLANTE,20913,adulto
45313,"Plastía de rebordes alveolares, paladar duro, ...",Retiro de implante (con osteotomía),ACCIONES IMPLANTOLOGIA,16049,adulto


In [30]:
CUSTOM_FILTERS = [
    lambda x: x.lower(),
    lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore'),
    strip_non_alphanum, 
    strip_punctuation, strip_multiple_whitespaces
    ]

df.provider_provision = df.provider_provision.map(lambda x: preprocess_string(x, CUSTOM_FILTERS)).map(lambda x: ' '.join(x))
df.provider_provision_specialty = df.provider_provision_specialty.map(lambda x: preprocess_string(x, CUSTOM_FILTERS)).map(lambda x: ' '.join(x))
df.edad = df.edad.map(lambda x: preprocess_string(x, CUSTOM_FILTERS)).map(lambda x: ' '.join(x))

In [31]:
df['code'] = '__label__' + df['code'].astype(str)

In [32]:
df.provider_provision = df.code + ' ' + df.provider_provision + ' ' + df.provider_provision_specialty + ' ' + df.edad
df.drop(columns=['code', 'edad', 'provider_provision_specialty', 'provision'], inplace=True)

In [33]:
df

,provider_provision
0,__label__35014 bc01004 urgencia tratamiento in...
1,__label__5066 blanqueamiento laser sin cobertu...
2,__label__5019 bc07031 recementacion incrustaci...
3,__label__5020 bc07032 recementacion de incrust...
4,__label__5028 bc07044 carillas directas operat...
...,...
45310,__label__6037 plural sobre implante 3 estructu...
45311,__label__6025 plural sobre implante 3 estructu...
45312,__label__20913 plural sobre implante 3 estruct...
45313,__label__16049 retiro de implante con osteotom...


In [34]:
df.to_csv('clasificaciones.ext.txt', index=False, sep='\t', header=False)

In [36]:
train, valid, test = np.split(df.sample(frac=1, random_state=42),  [int(.8*len(df)), int(.9*len(df))])
print(train.shape)
print(valid.shape)
print(test.shape)

(36252, 1)
(4531, 1)
(4532, 1)


In [37]:
train.to_csv('clasificaciones.train.ext.txt', index=False, sep=';', header=False)
valid.to_csv('clasificaciones.valid.ext.txt', index=False, sep=';', header=False)
test.to_csv('clasificaciones.test.ext.txt', index=False, sep=';', header=False)